<a href="https://colab.research.google.com/github/trancenoid/GANs/blob/master/audioGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir './tracks'


In [0]:
!cp './drive/My Drive/audioGAN_data/piano/A_h3kt6Dp9M' ./tracks/
!cp './drive/My Drive/audioGAN_data/piano/ET15VTJN-e0' ./tracks/
!cp './drive/My Drive/audioGAN_data/piano/pj1jHGU4a4g' ./tracks/
!cp './drive/My Drive/audioGAN_data/piano/3hr' ./tracks/

In [0]:
!cp ./tracks/A_h3kt6Dp9M './drive/My Drive/audioGAN_data/piano'
!cp ./tracks/ET15VTJN-e0 './drive/My Drive/audioGAN_data/piano'
!cp ./tracks/pj1jHGU4a4g './drive/My Drive/audioGAN_data/piano'

In [4]:
% pylab inline
import librosa
from librosa.display import waveplot,specshow
import os
import IPython.display as ipd
import pandas as pd
import glob 
from librosa.core import stft, istft
import keras
from keras.layers import *
from keras import initializers
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.models import Model, Sequential
from tqdm import tqdm
import numpy as np
import re


Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [5]:
print(os.listdir('./tracks'))
#!rmdir './tracks/.ipynb_checkpoints'

['A_h3kt6Dp9M', 'pj1jHGU4a4g', '3hr', 'ET15VTJN-e0']


In [6]:

#tracks = os.listdir('./drive/My Drive/audioGAN_data')
tracks = os.listdir('./tracks')
sr = 8191
clip_len = 8
#data, sampling_rate = librosa.load('016 - Timbaland ft. Keri Hilson - The Way I Are.MP3', sr = 8192)
def load_data():
  nt = len(tracks)
  X = []
  for track in tqdm(tracks):
    #data, _ = librosa.load('./drive/My Drive/audioGAN_data/' + track, sr = sr)
    data, _ = librosa.load('./tracks/' + track, sr = sr)
    data = [ np.array(stft(data[i:i + clip_len*sr]))[0:-1, :] for i in tqdm(range( int(len(data)/(clip_len*sr) + 1) ))]
    X.extend(data)
  data = np.array(X)
  print(data.shape)
  X = np.zeros(data.shape)
  Y = np.zeros(data.shape)
  for i in tqdm(range(data.shape[0])):
    for j in range(data.shape[1]):
      for k in range(data.shape[2]):
        #Y[i,j,k] = re.findall( '(-*[0-9]+\.[0-9]+)(?:[\+|\-])([0-9]+)', X[i,j,k])[0][0]
        temp = re.findall( '(-?[0-9]+\.?[0-9]+(e[\+|\-][0-9]+)?)?([\+|\-]?[0-9]+\.?[0-9]*(e[\+|\-][0-9]+)?)j',
                          str(data[i,j,k]))
        try:
          if(temp[0][0] != ''):
            X[i,j,k] = float(temp[0][0])
          else:
            X[i,j,k] = 0.0
          Y[i,j,k] = float(temp[0][2])
        except IndexError as e:
          print(data[i,j,k])
  np.save('./X', X)
  np.save('./Y', Y)
  !mv './X.npy' './drive/My Drive/audioGAN_data/piano/'
  !mv './Y.npy' './drive/My Drive/audioGAN_data/piano/'
#X = np.array(load_data()).transpose([1,2,3,0])
load_data()

100%|██████████| 4/4 [16:34<00:00, 219.19s/it]


(3041, 1024, 128)


100%|██████████| 3041/3041 [26:01<00:00,  2.02it/s]


NameError: ignored

In [0]:
!cp './drive/My Drive/audioGAN_data/piano/X.npy' './X.npy'
!cp './drive/My Drive/audioGAN_data/piano/Y.npy' './Y.npy'

In [0]:
!mv './data.npy' './drive/My Drive/audioGAN_data/piano/'

In [0]:
X = np.load('X.npy', mmap_mode = 'r')
Y = np.load('Y.npy', mmap_mode = 'r')

In [21]:
X = np.stack([X,Y]).transpose([1,2,3,0])
print(X.shape)
np.save('./X', X)

(3041, 1024, 128, 2)

In [0]:
np.save('./X', X)


In [0]:
!cp './X.npy' './drive/My Drive/audioGAN_data/piano/train'

In [0]:
!cp './drive/My Drive/audioGAN_data/piano/train' './data.npy'
X = np.load('data.npy', mmap_mode = 'r')


In [27]:
adam = Adam(lr = 0.0002, beta_1 = 0.5)
inShape = 100

def build_gen():
  
  G = Sequential()
  G.add(Dense(64*8*2, input_shape = (100,), activation = 'relu'))
  G.add(Reshape([64,8,2]))
  
  G.add(Conv2D(8,kernel_size = (5,3), strides = (1,1), padding = 'same'))
  #G.add(BatchNormalization(momentum = 0.9))
  G.add(LeakyReLU(0.1))
  
  G.add(Conv2DTranspose(16, strides = 2 ,kernel_size = (5,3), padding = 'same'))
  #G.add(BatchNormalization(momentum = 0.9))
  G.add(LeakyReLU(0.1))
  G.add(Dropout(0.3))
  
  G.add(Conv2D(16,kernel_size = (5,3), strides = (1,1), padding = 'same'))
  #G.add(BatchNormalization(momentum = 0.9))
  G.add(LeakyReLU(0.1))
  
  G.add(Conv2DTranspose(16, strides = 2 ,kernel_size = (5,3), padding = 'same'))
  #G.add(BatchNormalization(momentum = 0.9))
  G.add(LeakyReLU(0.1))
  G.add(Dropout(0.3))
  
  G.add(Conv2D(16,kernel_size = (5,3), strides = (1,1), padding = 'same'))
  #G.add(BatchNormalization(momentum = 0.9))
  G.add(LeakyReLU(0.1))
  
  G.add(Conv2DTranspose(16, strides = 2 ,kernel_size = (5,3), padding = 'same'))
  #G.add(BatchNormalization(momentum = 0.9))
  G.add(LeakyReLU(0.1))
  G.add(Dropout(0.3))
  
  G.add(Conv2D(16,kernel_size = (5,3), strides = (1,1), padding = 'same'))
  #G.add(BatchNormalization(momentum = 0.9))
  G.add(LeakyReLU(0.1))
  
  G.add(Conv2DTranspose(16, strides = 2 ,kernel_size = (5,3), padding = 'same'))
  #G.add(BatchNormalization(momentum = 0.9))
  G.add(LeakyReLU(0.1))
  G.add(Dropout(0.3))
  
  G.add(Conv2D(2, kernel_size = (5,3), padding = 'same', activation = 'tanh'))
  
  return G  
    

def build_dis():
  D = Sequential()
  D.add(Permute((2,1,3), input_shape = [1024,128,2,]))
  D.add(Reshape([128,1024*2]))
  D.add(LSTM(256 , activation = 'tanh'))
  D.add(Dense(128, activation = 'relu'))
  D.add(Dense(64, activation = 'relu'))
  D.add(Dense(1, activation = 'sigmoid'))
  D.compile(loss = 'binary_crossentropy', optimizer = adam)
  
  return D

D = build_dis()
G = build_gen()

D.trainable = False
ganIn = Input(shape = (inShape,))
genOut = G(ganIn)
ganOut = D(genOut)
GAN = Model(inputs = ganIn, outputs = ganOut)
GAN.compile(loss = 'binary_crossentropy', optimizer = adam)

W0708 07:19:59.655509 140513119209344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0708 07:19:59.660415 140513119209344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0708 07:19:59.719446 140513119209344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0708 07:20:00.173126 140513119209344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0708 07:20:00.179869 140513119209344 deprecation_wrappe

In [0]:
G.summary()

In [28]:
dLosses = []
gLosses = []
img_noise = np.random.rand(*X.shape)
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('dcgan_loss_epoch_%d.png' % epoch)
    
def generate_noise(n_samples, noise_dim):
  X = np.random.normal(0, 1, size=(n_samples, noise_dim))
  return X

def play_clips(batchidx):
  noise = generate_noise(5, 100)
  pred = G.predict(noise)
  ans = np.zeros((pred.shape[0], pred.shape[1], pred.shape[2]), dtype = complex)
  for i in range(pred.shape[0]):
    for j in range(pred.shape[1]):
      for k in range(pred.shape[2]):
        if(str(pred[i,j,k,1])[0] != '-'):
          ans[i,j,k] = complex(str(pred[i,j,k,0]) + '+' + str(pred[i,j,k,1]) + 'j')
        else:
          ans[i,j,k] = complex(str(pred[i,j,k,0]) + str(pred[i,j,k,1]) + 'j')

  for i in range(1):
    ipd.Audio(data = istft(ans[i]), rate = sr)
    


def saveModels(epoch):
  G.save('./drive/My Drive/audio_model/G_epoch_%d.h5' % epoch)
  D.save('./drive/My Drive/audio_model/D_epoch_%d.h5' % epoch)

def train(epochs=1, BATCH_SIZE=16):
  num_batches = int(X.shape[0]/BATCH_SIZE)
  N_EPOCHS = epochs
  for epoch in range(N_EPOCHS):

    cum_d_loss = 0.
    cum_g_loss = 0.

    for batch_idx in tqdm(range(num_batches)):
      clips = X[batch_idx*BATCH_SIZE : (batch_idx+1)*BATCH_SIZE]

      noise_data = generate_noise(BATCH_SIZE, 100)
      generated_clips = G.predict(noise_data)
      noise_prop = 0.05

      true_labels = np.zeros((BATCH_SIZE, 1)) + np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
      flipped_idx = np.random.choice(np.arange(len(true_labels)), size=int(noise_prop*len(true_labels)))
      true_labels[flipped_idx] = 1 - true_labels[flipped_idx]
      D.trainable = True 

      d_loss_true = D.train_on_batch(clips, true_labels)
      gene_labels = np.ones((BATCH_SIZE, 1)) - np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
      flipped_idx = np.random.choice(np.arange(len(gene_labels)), size=int(noise_prop*len(gene_labels)))
      gene_labels[flipped_idx] = 1 - gene_labels[flipped_idx]
      
    
      d_loss_gene = D.train_on_batch(generated_clips, gene_labels)
      D.trainable = False
      d_loss = 0.5 * np.add(d_loss_true, d_loss_gene)
      cum_d_loss += d_loss

      noise_data = generate_noise(BATCH_SIZE, 100)
      g_loss = GAN.train_on_batch(noise_data, np.zeros((BATCH_SIZE, 1)))
      cum_g_loss += g_loss
    print('  Epoch: {}, Generator Loss: {}, Discriminator Loss: {}'.format(epoch+1, cum_g_loss/num_batches, cum_d_loss/num_batches))
    if(epoch%5 == 0): 
      play_clips("epoch" + str(epoch))
      saveModels(epoch)

if __name__ == '__main__':
  train(5, 16)

100%|██████████| 190/190 [02:51<00:00,  1.19it/s]


  Epoch: 1, Generator Loss: 2.1236083805561066, Discriminator Loss: 0.3377255244474662


  0%|          | 0/190 [00:00<?, ?it/s]

  Epoch: 2, Generator Loss: 2.3848065062573083, Discriminator Loss: 0.2610653714129799


  0%|          | 0/190 [00:00<?, ?it/s]

  Epoch: 3, Generator Loss: 3.0405184374043817, Discriminator Loss: 0.24591557414908158


  0%|          | 0/190 [00:00<?, ?it/s]

  Epoch: 4, Generator Loss: 2.871781083157188, Discriminator Loss: 0.20904456047635328


100%|██████████| 190/190 [02:38<00:00,  1.20it/s]

  Epoch: 5, Generator Loss: 3.028267133863349, Discriminator Loss: 0.19929748012831336


In [29]:
pred = G.predict(np.random.normal(0, 1, size=(1, 100)))
ans = np.zeros((pred.shape[1], pred.shape[2]), dtype = complex)
print(D.predict(pred))
for i in range(pred[0].shape[0]):
    for j in range(pred[0].shape[1]):
      if(str(pred[0,i,j,1])[0] != '-'):
        #print(str(pred[0,i,j,0]) + '+' + str(pred[0,i,j,1])) 
        ans[i,j] = complex(str(pred[0,i,j,0]) + '+' + str(pred[0,i,j,1]) + 'j')
      else:
        #print(str(pred[0,i,j,0]) + str(pred[0,i,j,1])) 
        ans[i,j] = complex(str(pred[0,i,j,0]) + str(pred[0,i,j,1]) + 'j')
      
ans = istft(ans)
ipd.Audio(data = ans, rate = sr)

[[0.9330291]]


In [0]:
play_clips(1)